In [ ]:
!rm -rf /content/*

In [ ]:
import os
import pathlib
# Clone the tensorflow models repository if it doesn't already exist
if not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models
if not pathlib.Path('OD').exists(): 
  !git clone https://github.com/triasbrata/OD.git
!cd OD && git pull
if not pathlib.Path('images').exists(): 
  !cp -r /content/OD/images /content/images
if not pathlib.Path('training').exists(): 
  !cp -r /content/OD/training /content/training

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
#run model builder test
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

In [ ]:
import os
import pathlib
if not pathlib.Path('mobilenet_v2').exists():  
  %cd /content
  !wget http://download.tensorflow.org/models/object_detection/classification/tf2/20200710/mobilenet_v2.tar.gz
  !tar -xvf mobilenet_v2.tar.gz
  !rm mobilenet_v2.tar.gz
  

In [ ]:
!wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_mobilenet_v2_320x320_coco17_tpu-8.config
!mv ssd_mobilenet_v2_320x320_coco17_tpu-8.config mobilenet_v2.config

In [ ]:
num_classes = 1
batch_size = 96
num_steps = 7500
num_eval_steps = 1000

labelmap_path = "/content/training/labelmap.pbtxt"
test_record_path ="/content/training/test.record"
train_record_path="/content/training/train.record"

model_dir = '/content/training/'

pipeline_config_path = '/content/model_config.config'
fine_tune_checkpoint = '/content/mobilenet_v2/mobilenet_v2.ckpt-1'

image_folder="/content/images/"

In [ ]:
!python OD/xml_to_csv.py {image_folder}

In [ ]:
!python OD/generate_tfrecord.py --csv_input=/content/images/train_labels.csv --image_dir=/content/images/train --output_path={train_record_path}
!python OD/generate_tfrecord.py --csv_input=/content/images/test_labels.csv --image_dir=/content/images/test --output_path={test_record_path}

In [ ]:
# edit configuration file (from https://colab.research.google.com/drive/1sLqFKVV94wm-lglFq_0kGo2ciM0kecWD)

import re

with open(base_config_path) as f:
    config = f.read()

with open('model_config.config', 'w') as f:
  
  
  # Set labelmap path
  config = re.sub('label_map_path: ".*?"', 
             'label_map_path: "{}"'.format(labelmap_path), config)
  
  # Set fine_tune_checkpoint path
  config = re.sub('fine_tune_checkpoint: ".*?"',
                  'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), config)
  
  # Set train tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 
                  'input_path: "{}"'.format(train_record_path), config)
  
  # Set test tf-record file path
  config = re.sub('(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 
                  'input_path: "{}"'.format(test_record_path), config)
  
  # Set number of classes.
  config = re.sub('num_classes: [0-9]+',
                  'num_classes: {}'.format(num_classes), config)
  
  # Set batch size
  config = re.sub('batch_size: [0-9]+',
                  'batch_size: {}'.format(batch_size), config)
  
  # Set training steps
  config = re.sub('num_steps: [0-9]+',
                  'num_steps: {}'.format(num_steps), config)
  
  f.write(config)

In [ ]:
%cat {pipeline_config_path}

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_config_path} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps}

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_config_path} \
    --model_dir={model_dir} \
    --checkpoint_dir={model_dir}  

In [ ]:
%load_ext tensorboard
%tensorboard --logdir '/content/training/'

In [ ]:
with open('/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/utils/tf_utils.py') as f:
    tf_utils = f.read()

with open('/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/utils/tf_utils.py', 'w') as f:
  # Set labelmap path
  throw_statement = "raise TypeError('Expected Operation, Variable, or Tensor, got ' + str(x))"
  tf_utils = tf_utils.replace(throw_statement, "if not isinstance(x, str):" + throw_statement)
  f.write(tf_utils)



In [ ]:
output_directory = 'inference_graph'

!python /content/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir {model_dir} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_config_path}

In [ ]:
!zip -r /content/saved_model.zip /content/inference_graph/saved_model/

In [ ]:
from google.colab import files
files.download("/content/saved_model.zip")

In [ ]:
output_directory = 'inference_graph'

!python /content/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir {model_dir} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_config_path}

In [ ]:
!zip -r /content/saved_model.zip /content/inference_graph/saved_model/
!zip -r /content/training.zip /content/training/

In [ ]:
from google.colab import files
files.download("/content/saved_model.zip")
files.download("/content/training.zip")

In [ ]:
output_directory = 'inference_graph/'

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(labelmap_path, use_display_name=True)
tf.keras.backend.clear_session()
model = tf.saved_model.load(f'/content/{output_directory}/saved_model')

In [ ]:
import pandas as pd
test = pd.read_csv('/content/images/test_labels.csv')
#Getting 3 random images to test
images = list(test.sample(n=15)['filename'])

In [ ]:
for image_name in images:
  
  image_np = load_image_into_numpy_array('/content/images/test/' + image_name)
  output_dict = run_inference_for_single_image(model, image_np)
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)
  display(Image.fromarray(image_np))